# Gráficos computacionales y tf.function

* 30:00 min | Última modificación: Mayo 10, 2021 | [YouTube]

Adaptado de:

* https://www.tensorflow.org/tutorials/keras/keras_tuner

In [1]:
import tensorflow as tf

#  import timeit
#  from datetime import datetime

## Graficos computacionales y funciones en Python

In [2]:
#
# Función en Python que usa TF internamente
#
def a_regular_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x


#
# Convierte la función en Python a una
# función de TensorFlow
#
a_function_that_uses_a_graph = tf.function(a_regular_function)


#
# Cómputos
#
x1 = tf.constant(
    [
        [1.0, 2.0],
    ]
)
y1 = tf.constant(
    [
        [2.0],
        [3.0],
    ]
)
b1 = tf.constant(4.0)

#
# Llama a la función escrita en Python
#
orig_value = a_regular_function(x1, y1, b1).numpy()

#
# Llama a la función escrita en TensorFlow
#
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()

orig_value, tf_function_value

(array([[12.]], dtype=float32), array([[12.]], dtype=float32))

In [3]:
def inner_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x


#
# El decorador convierte la función en Python en
# una función en TensorFlow
#
@tf.function
def outer_function(x):
    y = tf.constant([[2.0], [3.0]])
    b = tf.constant(4.0)
    return inner_function(x, y, b)


#
# Internamente crea el gráfico computacional
#  que incluye inner_function
#
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

## Transformación de funciones de Python a gráficas computacionales

In [4]:
#
# Función de activación ReLU
#
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0


#
# Creación de la gráfica computacional
#
tf_simple_relu = tf.function(simple_relu)

tf_simple_relu(tf.constant(1)).numpy(), tf_simple_relu(tf.constant(-1)).numpy()

(1, 0)

In [5]:
#
# Imprime el código generado por tf.function
#
print(tf.autograph.to_code(simple_relu))

def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_bo

In [6]:
#
# Imprime la gráfica computacional
#
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "Tcond"
    value {
      type: DT_BOOL
    }
  }
  attr {
    key: "Tin"
    value {
      list {
        type: DT_INT32
      }
    }
  }
  attr {
    key: "Tout"
    value {
      list {
        type: DT_BOOL
        type: DT_INT32
      }
    }
  

## Polimorfismo

In [7]:
@tf.function
def my_relu(x):
    return tf.maximum(0.0, x)


#
# Llama la función con distintos tipos de datos
#
print(my_relu(tf.constant(5.5)))
print(my_relu([1, -1]))
print(my_relu(tf.constant([3.0, -3.0])))

tf.Tensor(5.5, shape=(), dtype=float32)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor([3. 0.], shape=(2,), dtype=float32)


## tf.function

In [8]:
@tf.function
def get_MSE(y_true, y_pred):
    sq_diff = tf.pow(y_true - y_pred, 2)
    return tf.reduce_mean(sq_diff)


y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)

y_true.numpy(), y_pred.numpy()

(array([1, 4, 5, 0, 3], dtype=int32), array([3, 5, 8, 2, 8], dtype=int32))

In [9]:
#
# Cómputo del MSE
#
get_MSE(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=8>